# Training Model
Keras on Tensorflow

trained from  /2020_01_24/model_2020_01_24_16_02.h5

In [1]:
import os, sys
import cv2
import numpy as np
import uuid
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from imutils import paths
import itertools
import json
from pprint import pprint



import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from core.imageprep import dir_checker, random_crop, crop_generator, random_crop_batch
from core.models import UNet, UNet_hp, vanilla_unet, vanilla_unet_nodrop
from core.metrics import iou_coef, dice_coef

from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorboard.plugins.hparams import api as hp
from datetime import datetime

from IPython import get_ipython
# %load_ext autoreload
get_ipython().run_line_magic('load_ext', 'autoreload')
# %autoreload 2
get_ipython().run_line_magic('autoreload', '2')
# %load_ext tensorboard
get_ipython().run_line_magic('load_ext', 'tensorboard')

In [2]:
from packaging import version
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [3]:
sys.executable

'C:\\Users\\wucci_admin\\Anaconda3\\envs\\tfdl02\\python.exe'

 ## Load Training Dataset

In [4]:
# load image
print("Load Images...")
# on mac
# path = "/Volumes/LaCie_DataStorage/PerlmutterData/"

# on Window PC 
path = os.path.join('D:', 'PerlmutterData')

# input set
crop_input_set = '2020_01_23_09_51_20x'
# crop_input_set = '2020_01_30_11_24_1x' # small training set

imginput = os.path.join('dl_seg_project_raw', 'data_crop', crop_input_set,)
imgpath = os.path.join(path, imginput)

print(imgpath)

img_dir = os.path.join(imgpath, 'images')
label_dir = os.path.join(imgpath, 'labels')
print(img_dir)
print(label_dir)

# check if the output folder exist. If not, create a folder
dir_checker('logs', path)
path_logs = os.path.join(path, 'logs')
dir_checker('fit', path_logs)
dir_checker('model', path_logs)
dir_checker('pars', path_logs)

Load Images...
D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x
D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x\images
D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x\labels
logs exists in D:PerlmutterData
fit exists in D:PerlmutterData\logs
model exists in D:PerlmutterData\logs
pars exists in D:PerlmutterData\logs


 Print the first file name.

In [5]:
imgpath_all = list(paths.list_images(imgpath))
print(imgpath_all[0])

D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x\images\nucleus\0001.tif


 ## Create Image Datagenerator
 1. create only one datagen
 2. specify valiation split in datagen argument
 3. add split data when calling `datagen.flow_from_directory`

In [6]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
date =  datetime.now().strftime("%Y_%m_%d")
seed = 102
batch_size = 16
epoch = 500
validation_steps = 20
validation_split = 0.3
training_sample_size = len(imgpath_all)
IMG_HEIGHT = None
IMG_WIDTH = None
classes = ['cell_membrane', 'nucleus', 'autophagosome']
inputclass = [classes[1]]
learning_rate = 1e-6
loss = "binary_crossentropy"
metrics = ['accuracy', iou_coef, dice_coef]

metrics_name = []
for f in metrics:
    if callable(f):
        metrics_name.append(f.__name__)
    else:
        metrics_name.append(f)

In [7]:
# create arguments for data generator
data_gen_img_args = dict(
                # featurewise_center = True,
                # featurewise_std_normalization = True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                rescale=1.0/255.0,
                )

data_gen_label_args = dict(
                # featurewise_center=True,
                # featurewise_std_normalization=True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                # rescale=1.0/255.0,
                rescale=1.0/255.0,
                )

# create parameter
pars = dict(
                # basic information
                timestamp = timestamp,
                date = date,
                seed = seed,
                batch_size = batch_size,
                
                # Data generator
                crop_input_set = crop_input_set,
                validation_steps = validation_steps,
                validation_split = validation_split,
                training_sample_size = training_sample_size,
                
                # training class
                classes = classes,
                inputclass = inputclass,
    
                # add datagen args
                data_gen_img_args = data_gen_img_args,
                data_gen_label_args = data_gen_label_args,
                
                # Build model
                IMG_HEIGHT = IMG_HEIGHT,
                IMG_WIDTH = IMG_WIDTH,
                epoch = epoch, 
                loss = loss,
                metrics_name = metrics_name,
                learning_rate = learning_rate,
                )

# save parameter
path_pars = os.path.join(path_logs, 'pars')
dir_checker(inputclass[0], path_pars)
dir_checker(date, os.path.join(path_pars, inputclass[0]))

pprint(pars)

nucleus exists in D:PerlmutterData\logs\pars
2020_02_04 exists in D:PerlmutterData\logs\pars\nucleus
{'IMG_HEIGHT': None,
 'IMG_WIDTH': None,
 'batch_size': 16,
 'classes': ['cell_membrane', 'nucleus', 'autophagosome'],
 'crop_input_set': '2020_01_23_09_51_20x',
 'data_gen_img_args': {'height_shift_range': 0.1,
                       'horizontal_flip': True,
                       'rescale': 0.00392156862745098,
                       'rotation_range': 90.0,
                       'shear_range': 0.07,
                       'validation_split': 0.3,
                       'vertical_flip': True,
                       'width_shift_range': 0.1,
                       'zoom_range': 0.2},
 'data_gen_label_args': {'height_shift_range': 0.1,
                         'horizontal_flip': True,
                         'rescale': 0.00392156862745098,
                         'rotation_range': 90.0,
                         'shear_range': 0.07,
                         'validation_split': 0.3,
   

In [8]:
par_file_dir = os.path.join(path_pars, inputclass[0], date, 'pars_' + timestamp + '.json')
print(par_file_dir)

with open(par_file_dir, 'w') as outfile:
    json.dump(pars, outfile, indent=4)


D:PerlmutterData\logs\pars\nucleus\2020_02_04\pars_2020_02_04_17_08.json


In [9]:
# create generator
image_datagen = ImageDataGenerator(**data_gen_img_args)
label_datagen = ImageDataGenerator(**data_gen_label_args)

In [10]:
# load images into generator
train_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training', # <- define subset as 'training'
    seed=seed)

train_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training',
    seed=seed)

valid_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation', # <- define subset as 'validation'
    seed=seed)

valid_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation',
    seed=seed)

Found 4634 images belonging to 1 classes.
Found 4634 images belonging to 1 classes.
Found 1986 images belonging to 1 classes.
Found 1986 images belonging to 1 classes.


In [11]:

# merge image and label generator
def combine_generator(gen1, gen2):
    while True:
        yield(gen1.next(), gen2.next()) 
train_generator = combine_generator(train_image_generator, train_label_generator)
valid_generator = combine_generator(valid_image_generator, valid_label_generator)


In [12]:
'''
train_generator = zip(train_image_generator, train_label_generator)
valid_generator = zip(valid_image_generator, valid_label_generator)
'''

'\ntrain_generator = zip(train_image_generator, train_label_generator)\nvalid_generator = zip(valid_image_generator, valid_label_generator)\n'

 ## Training

In [13]:
print("Start training...")

Start training...


 ### Define Callbacks

In [14]:
# checkpoint
modelfilename = 'model_' + timestamp + '.h5'
dir_checker(inputclass[0], os.path.join(path_logs, 'model'))
dir_checker(date, os.path.join(path_logs, 'model', inputclass[0]))
modelfile_path = os.path.join(path_logs, 'model', inputclass[0], date, modelfilename)

checkpointer = ModelCheckpoint(filepath = modelfile_path, 
                               monitor = 'val_loss', 
                               mode = 'min', 
                               verbose = 1, 
                               save_best_only = True)

# early stopping 
early_stopping = EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4)

# learning rate adjustment
reduceLR = ReduceLROnPlateau(monitor='val_loss',
                    factor=0.1,
                    patience=4,
                    verbose=1,
                    min_delta=1e-4)

# tensorboard
from tensorflow.summary import create_file_writer

dir_checker(inputclass[0], os.path.join(path_logs, 'fit'))
dir_checker(date, os.path.join(path_logs, 'fit', inputclass[0]))
logdir = os.path.join(path_logs, 'fit', inputclass[0], date, timestamp)

file_writer = create_file_writer(logdir + "/metrics")
file_writer.set_as_default()

metrics = ['accuracy', iou_coef, dice_coef]


tensorboard_callback = TensorBoard(log_dir = logdir, 
                                   profile_batch = 0, 
                                   update_freq= 500,
                                   histogram_freq = 1
                                   )

# compile callbacks
# callbacks = [checkpointer, tensorboard_callback, early_stopping, reduceLR]
callbacks = [checkpointer, tensorboard_callback, reduceLR]

nucleus exists in D:PerlmutterData\logs\model
2020_02_04 exists in D:PerlmutterData\logs\model\nucleus
nucleus exists in D:PerlmutterData\logs\fit
2020_02_04 exists in D:PerlmutterData\logs\fit\nucleus


 ### Setup the model

In [15]:
# calculate steps_per_epoch
steps_per_epoch = training_sample_size * (1 - validation_split) // batch_size
print("Steps per epoch: {}".format(steps_per_epoch))

Steps per epoch: 579.0


In [16]:
print(logdir)

D:PerlmutterData\logs\fit\nucleus\2020_02_04\2020_02_04_17_08


In [17]:
'''
# prepare the model
unetmodel = UNet(shape = (IMG_HEIGHT, IMG_WIDTH), 
                 lr = learning_rate, 
                 loss = loss,
                 metrics = metrics,
                )

# train the model
unetmodel.fit_generator(
                    generator = train_generator, 
                    validation_data = valid_generator,
                    validation_steps = validation_steps,
                    steps_per_epoch = steps_per_epoch,
                    epochs = epoch,  
                    callbacks = callbacks,
                    verbose = 1, 
                    )
'''

'\n# prepare the model\nunetmodel = UNet(shape = (IMG_HEIGHT, IMG_WIDTH), \n                 lr = learning_rate, \n                 loss = loss,\n                 metrics = metrics,\n                )\n\n# train the model\nunetmodel.fit_generator(\n                    generator = train_generator, \n                    validation_data = valid_generator,\n                    validation_steps = validation_steps,\n                    steps_per_epoch = steps_per_epoch,\n                    epochs = epoch,  \n                    callbacks = callbacks,\n                    verbose = 1, \n                    )\n'

In [18]:
from tensorflow.keras.models import load_model

path_model = os.path.join(os.path.join(path_logs, 'model', inputclass[0]), '2020_02_04', 'model_2020_02_04_16_15.h5')

dropout = 0.7
num_layers = 4

unetmodel = vanilla_unet_nodrop(
                            shape = (IMG_HEIGHT, IMG_WIDTH), 
                            dropout = dropout, 
                            num_layers = num_layers, 
                            lr = learning_rate, 
                            loss = loss,
                            metrics = metrics,
                            summary = True,
                            use_batch_norm = False
                           )

unetmodel.load_weights(path_model)

unetmodel.fit_generator(
                    generator = train_generator, 
                    validation_data = valid_generator,
                    validation_steps = validation_steps,
                    steps_per_epoch = steps_per_epoch,
                    epochs = epoch,  
                    callbacks = callbacks,
                    verbose = 1, 
                    )

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 6 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 6 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, None, None, 6 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

578/579 [============================>.] - ETA: 0s - loss: 0.1387 - accuracy: 0.9556 - iou_coef: 0.3134 - dice_coef: 0.3377
Epoch 00001: val_loss improved from inf to 0.28046, saving model to D:PerlmutterData\logs\model\nucleus\2020_02_04\model_2020_02_04_17_08.h5
579/579 [==============================] - 509s 878ms/step - loss: 0.1392 - accuracy: 0.9554 - iou_coef: 0.3131 - dice_coef: 0.3374 - val_loss: 0.2805 - val_accuracy: 0.9124 - val_iou_coef: 0.4209 - val_dice_coef: 0.4608
Epoch 2/500
578/579 [============================>.] - ETA: 0s - loss: 0.1364 - accuracy: 0.9566 - iou_coef: 0.3128 - dice_coef: 0.3371
Epoch 00002: val_loss improved from 0.28046 to 0.22957, saving model to D:PerlmutterData\logs\model\nucleus\2020_02_04\model_2020_02_04_17_08.h5
579/579 [==============================] - 499s 862ms/step - loss: 0.1368 - accuracy: 0.9565 - iou_coef: 0.3130 - dice_coef: 0.3373 - val_loss: 0.2296 - val_accuracy: 0.9185 - val_iou_coef: 0.4186 - val_dice_coef: 0.4646
Epoch 3/500


578/579 [============================>.] - ETA: 0s - loss: 0.1276 - accuracy: 0.9580 - iou_coef: 0.3185 - dice_coef: 0.3423
Epoch 00019: val_loss did not improve from 0.19470

Epoch 00019: ReduceLROnPlateau reducing learning rate to 9.999999717180686e-11.
579/579 [==============================] - 495s 855ms/step - loss: 0.1275 - accuracy: 0.9580 - iou_coef: 0.3187 - dice_coef: 0.3425 - val_loss: 0.2985 - val_accuracy: 0.8977 - val_iou_coef: 0.4368 - val_dice_coef: 0.4848
Epoch 20/500
578/579 [============================>.] - ETA: 0s - loss: 0.1295 - accuracy: 0.9575 - iou_coef: 0.3141 - dice_coef: 0.3378
Epoch 00020: val_loss did not improve from 0.19470
579/579 [==============================] - 495s 855ms/step - loss: 0.1294 - accuracy: 0.9575 - iou_coef: 0.3143 - dice_coef: 0.3380 - val_loss: 0.2519 - val_accuracy: 0.9123 - val_iou_coef: 0.4252 - val_dice_coef: 0.4711
Epoch 21/500
578/579 [============================>.] - ETA: 0s - loss: 0.1281 - accuracy: 0.9574 - iou_coef: 0.31

578/579 [============================>.] - ETA: 0s - loss: 0.1270 - accuracy: 0.9588 - iou_coef: 0.3146 - dice_coef: 0.3382
Epoch 00038: val_loss did not improve from 0.19470
579/579 [==============================] - 494s 854ms/step - loss: 0.1269 - accuracy: 0.9588 - iou_coef: 0.3148 - dice_coef: 0.3384 - val_loss: 0.2464 - val_accuracy: 0.9109 - val_iou_coef: 0.4508 - val_dice_coef: 0.4951
Epoch 39/500
578/579 [============================>.] - ETA: 0s - loss: 0.1307 - accuracy: 0.9565 - iou_coef: 0.3145 - dice_coef: 0.3383
Epoch 00039: val_loss did not improve from 0.19470

Epoch 00039: ReduceLROnPlateau reducing learning rate to 9.999998977483753e-16.
579/579 [==============================] - 494s 853ms/step - loss: 0.1309 - accuracy: 0.9565 - iou_coef: 0.3145 - dice_coef: 0.3383 - val_loss: 0.2647 - val_accuracy: 0.9112 - val_iou_coef: 0.4067 - val_dice_coef: 0.4494
Epoch 40/500
578/579 [============================>.] - ETA: 0s - loss: 0.1303 - accuracy: 0.9566 - iou_coef: 0.31

578/579 [============================>.] - ETA: 0s - loss: 0.1294 - accuracy: 0.9569 - iou_coef: 0.3166 - dice_coef: 0.3407
Epoch 00057: val_loss did not improve from 0.15303

Epoch 00057: ReduceLROnPlateau reducing learning rate to 9.999999424161285e-20.
579/579 [==============================] - 495s 854ms/step - loss: 0.1294 - accuracy: 0.9568 - iou_coef: 0.3168 - dice_coef: 0.3408 - val_loss: 0.2317 - val_accuracy: 0.9171 - val_iou_coef: 0.4159 - val_dice_coef: 0.4598
Epoch 58/500
578/579 [============================>.] - ETA: 0s - loss: 0.1269 - accuracy: 0.9580 - iou_coef: 0.3117 - dice_coef: 0.3349
Epoch 00058: val_loss did not improve from 0.15303
579/579 [==============================] - 495s 855ms/step - loss: 0.1268 - accuracy: 0.9580 - iou_coef: 0.3118 - dice_coef: 0.3350 - val_loss: 0.2450 - val_accuracy: 0.9153 - val_iou_coef: 0.4488 - val_dice_coef: 0.4941
Epoch 59/500
578/579 [============================>.] - ETA: 0s - loss: 0.1316 - accuracy: 0.9565 - iou_coef: 0.31

578/579 [============================>.] - ETA: 0s - loss: 0.1300 - accuracy: 0.9567 - iou_coef: 0.3186 - dice_coef: 0.3422
Epoch 00076: val_loss did not improve from 0.15303
579/579 [==============================] - 495s 855ms/step - loss: 0.1298 - accuracy: 0.9567 - iou_coef: 0.3185 - dice_coef: 0.3421 - val_loss: 0.2601 - val_accuracy: 0.9100 - val_iou_coef: 0.4044 - val_dice_coef: 0.4526
Epoch 77/500
578/579 [============================>.] - ETA: 0s - loss: 0.1313 - accuracy: 0.9567 - iou_coef: 0.3121 - dice_coef: 0.3360
Epoch 00077: val_loss did not improve from 0.15303

Epoch 00077: ReduceLROnPlateau reducing learning rate to 9.999999998199588e-25.
579/579 [==============================] - 495s 855ms/step - loss: 0.1312 - accuracy: 0.9567 - iou_coef: 0.3119 - dice_coef: 0.3357 - val_loss: 0.2274 - val_accuracy: 0.9175 - val_iou_coef: 0.4195 - val_dice_coef: 0.4628
Epoch 78/500
578/579 [============================>.] - ETA: 0s - loss: 0.1255 - accuracy: 0.9589 - iou_coef: 0.31

578/579 [============================>.] - ETA: 0s - loss: 0.1276 - accuracy: 0.9579 - iou_coef: 0.3159 - dice_coef: 0.3396
Epoch 00095: val_loss did not improve from 0.15303
579/579 [==============================] - 494s 853ms/step - loss: 0.1275 - accuracy: 0.9579 - iou_coef: 0.3158 - dice_coef: 0.3396 - val_loss: 0.3122 - val_accuracy: 0.9092 - val_iou_coef: 0.4327 - val_dice_coef: 0.4800
Epoch 96/500
578/579 [============================>.] - ETA: 0s - loss: 0.1302 - accuracy: 0.9573 - iou_coef: 0.3150 - dice_coef: 0.3385
Epoch 00096: val_loss did not improve from 0.15303
579/579 [==============================] - 494s 853ms/step - loss: 0.1301 - accuracy: 0.9574 - iou_coef: 0.3155 - dice_coef: 0.3390 - val_loss: 0.2584 - val_accuracy: 0.9094 - val_iou_coef: 0.4463 - val_dice_coef: 0.4928
Epoch 97/500
578/579 [============================>.] - ETA: 0s - loss: 0.1306 - accuracy: 0.9567 - iou_coef: 0.3168 - dice_coef: 0.3405
Epoch 00097: val_loss did not improve from 0.15303

Epoch 

KeyboardInterrupt: 